In [1]:
from ImageNet import *

db_path = "/data/lab_ietr/hbrachem/ImageNet/"
dataset = ImageNet_dataset(os.path.join(db_path,"serialized_val.pickle"),db_path=os.path.join(db_path,"ILSVRC/Data/CLS-LOC/val/"))

In [2]:
from model_utils import *

vlm = 'Llava-7b'
model,processor = vlm_pretrained(vlm)
device = model.device

2024-09-10 10:17:01.210077: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-10 10:17:01.225706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 10:17:01.248142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 10:17:01.248174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 10:17:01.262732: I tensorflow/core/platform/cpu_feature_gua

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home-reseau/hbrachem/miniconda3/envs/py311env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
encoder = encoder_QKV(vlm,model)
w = processor.image_processor.crop_size['width']
h = processor.image_processor.crop_size['height']
patch_dim = 14

In [9]:
num_targeted_layer = 2

attack_config = {
    'lr':"1e-3",
    'component': 'encoder',
    'layers':f"{num_targeted_layer}F",
    'bounding_box':'scaled*336',
    'algo':  "min_att",
    'loss':"att_mean",
    'optimizer':'adam',
    'steps':1000,
    'checkpoint':250,
}

target_layers_v = encoder["V"][:num_targeted_layer]
target_layers_k = encoder["K"][:num_targeted_layer]
target_layers_q = encoder["Q"][:num_targeted_layer]

In [10]:
path = f"ablation_study/llava_{attack_config['component']}_{attack_config['layers']}/algo_{attack_config['algo']}/{attack_config['loss']}/{attack_config['optimizer']}_{attack_config['lr']}"

!mkdir -p $path
folder = path+'/adv_img'
!mkdir $folder
folder = path+'/hist'
!mkdir $folder
folder = path+'/predictions'
!mkdir $folder
folder = path+'/best'
!mkdir $folder

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory ‘ablation_study/llava_encoder_2F/algo_min_att/att_mean/adam_1e-3/adv_img’: File exists
mkdir: cannot create directory ‘ablation_study/llava_encoder_2F/algo_min_att/att_mean/adam_1e-3/hist’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...


mkdir: cannot create directory ‘ablation_study/llava_encoder_2F/algo_min_att/att_mean/adam_1e-3/predictions’: File exists


To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


mkdir: cannot create directory ‘ablation_study/llava_encoder_2F/algo_min_att/att_mean/adam_1e-3/best’: File exists


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
from attack_utils import *

In [12]:
import numpy as np

In [ ]:
for i ,[image,label,boxes,img_name] in enumerate(dataset):
    print(f"Processing sample number {i}...")
    if i == 50:
        break
        
    image = image.convert('RGB').resize((w,h))
    ref = np.array(image.convert('RGB')).copy()
    boxes = bounding_box_new_position(image,boxes,w,h)

    parametters = {'image':image}
    parametters['label'] = label
    parametters['boxes'] = boxes
    parametters['model'] = model
    parametters['processor'] = processor
    parametters['optimizer'] = attack_config["optimizer"]
    parametters['lr'] = float(attack_config['lr'])
    parametters['target_layers_q'] = target_layers_q
    parametters['target_layers_k'] = target_layers_k
    parametters['target_layers_v'] = target_layers_v
    parametters['lambda_a'] = 1
    parametters['lambda_e'] = 0
    parametters['lambda_n'] = 0
    parametters['w'] = w
    parametters['h'] = h
    parametters['patch_dim'] = patch_dim
    parametters['steps'] = attack_config["steps"]
    parametters['checkpoint'] = attack_config["checkpoint"]
    parametters['path'] = path
    parametters['img_name'] = img_name
    
    best_image, exec_time = generate_adv_image(**parametters)

    print(f"execution_time: {exec_time}")
    
    
    

Processing sample number 0...


 10%|▉         | 95/1000 [02:26<23:19,  1.55s/it]